In [ ]:
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Clone NHÁNH convnext-v2-dev
if not os.path.exists('/content/FR_Photometric_Stereo'):
    !git clone -b convnext-v2-dev https://github.com/minh4923/FR_Photometric_Stereo.git

# 3. Di chuyển vào thư mục dự án
%cd /content/FR_Photometric_Stereo

# 4. Cài đặt thư viện
!pip install -q albumentations==1.3.1 timm


Mounted at /content/drive
Cloning into 'FR_Photometric_Stereo'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 142 (delta 48), reused 132 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (142/142), 10.79 MiB | 32.32 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/FR_Photometric_Stereo
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
%cd /content/FR_Photometric_Stereo
import warnings
warnings.filterwarnings("ignore")
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A
import os
import pandas as pd

# Import các module đã sửa đổi
from going_modular.dataloader.multitask import create_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.loss.MultiTaskLoss import MultiTaskLoss
from going_modular.train_eval.train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.ExperimentManager import ExperimentManager

# --- CẤU HÌNH ---
device = "cuda" if torch.cuda.is_available() else "cpu"
seed = 42
torch.manual_seed(seed)


EXPERIMENT_NAME = "Single_ALBEDO_PK_SAMPLER_ConvNextV2"

CONFIGURATION = {
    'note': EXPERIMENT_NAME,
    'dataset_dir': '/content/drive/MyDrive/Photometric_DB_Full/',


    'type': 'albedo',

    'backbone': 'convnextv2_tiny',
    'use_sampler': True,

    'device': device,
    'epochs': 100,
    'batch_size': 16,
    'image_size': 112,
    'base_lr': 1e-4,
    'num_classes': None,

    # Trọng số loss
    'loss_gender_weight': 1.0,      'loss_da_gender_weight': 0.1,
    'loss_emotion_weight': 0.5,     'loss_da_emotion_weight': 0.1,
    'loss_pose_weight': 1.0,        'loss_da_pose_weight': 0.1,
    'loss_spectacles_weight': 0.5,  'loss_da_spectacles_weight': 0.1,
    'loss_facial_hair_weight': 0.5, 'loss_da_facial_hair_weight': 0.1,
}

# --- 1. KHỞI TẠO MANAGER ---
manager = ExperimentManager(CONFIGURATION)
manager.log_text(f"TRAINING MODE: {CONFIGURATION['type']} | Backbone: {CONFIGURATION['backbone']} | PK Sampler: {CONFIGURATION['use_sampler']}")

# --- 2. DATASET ---
train_csv_path = os.path.join(CONFIGURATION['dataset_dir'], 'dataset/train_split.csv')
if not os.path.exists(train_csv_path): train_csv_path = os.path.join(CONFIGURATION['dataset_dir'], 'train_split.csv')


df_temp = pd.read_csv(train_csv_path)
CONFIGURATION['num_classes'] = df_temp['id'].max() + 1
manager.log_text(f"Max ID in CSV: {df_temp['id'].max()} -> Set num_classes: {CONFIGURATION['num_classes']}")

train_transform = A.Compose([
    RandomResizedCropRect(CONFIGURATION['image_size']),
    GaussianNoise(p=0.2),
    A.HorizontalFlip(p=0.5)
], additional_targets={'albedo': 'image', 'depthmap': 'image', 'normal': 'image'})

test_transform = A.Compose([
    A.Resize(CONFIGURATION['image_size'], CONFIGURATION['image_size'])
], additional_targets={'albedo': 'image', 'depthmap': 'image', 'normal': 'image'})

train_dl, test_dl, _ = create_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)


model = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])

criterion = MultiTaskLoss(train_csv_path, CONFIGURATION)
optimizer = Adam(model.parameters(), lr=CONFIGURATION['base_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2)

# --- 4. CHECKPOINT SAVER ---
ckpt_saver = ModelCheckpoint(
    output_dir=manager.ckpt_dir,
    mode='max',
    best_metric_name='auc_id_cosine'
)

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc'], patience=1500, mode='max', verbose=0,
    save_dir=manager.ckpt_dir, start_from_epoch=0
)

# --- 5. RUN ---
print(f" Bắt đầu train Single Task: {CONFIGURATION['type'].upper()} với ConvNeXt V2...")
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dl,
    test_dataloader=test_dl,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    early_stopping=early_stopping,
    model_checkpoint=ckpt_saver,
    existing_manager=manager
)

/content/FR_Photometric_Stereo
KHOI TAO THI NGHIEM: Single_ALBEDO_PK_SAMPLER_ConvNextV2
Luu tru tai: /content/drive/MyDrive/Photometric_DB_Full/experiments/Single_ALBEDO_PK_SAMPLER_ConvNextV2
Thoi gian: 2026-01-11 09:47:21
--------------------------------------------------
TRAINING MODE: albedo | Backbone: convnextv2_tiny | PK Sampler: True
Max ID in CSV: 2231 -> Set num_classes: 2232
>>> SingleLoader: MODE = PK SAMPLER
--- Loading ConvNeXt V2: convnextv2_tiny (Pretrained=True) ---


model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

 Bắt đầu train Single Task: ALBEDO với ConvNeXt V2...
BAT DAU TRAINING: Single_ALBEDO_PK_SAMPLER_ConvNextV2

--- Epoch 1/100 ---
Ep 1:
╒═════════════════════╤═══════════╤════════════════════╕
│ Metric              │     Train │ Test               │
╞═════════════════════╪═══════════╪════════════════════╡
│ loss                │ 22.0599   │ -                  │
├─────────────────────┼───────────┼────────────────────┤
│ loss_id             │ 31.8339   │ -                  │
├─────────────────────┼───────────┼────────────────────┤
│ loss_gender         │  0.471967 │ 0.3877628768483798 │
├─────────────────────┼───────────┼────────────────────┤
│ loss_da_gender      │  1.78494  │ 1.7707035210397508 │
├─────────────────────┼───────────┼────────────────────┤
│ loss_emotion        │  0.54852  │ 0.5516374632716179 │
├─────────────────────┼───────────┼────────────────────┤
│ loss_da_emotion     │  1.43069  │ 2.1038650737868414 │
├─────────────────────┼───────────┼────────────────────┤
│ loss_pos